<a href="https://colab.research.google.com/github/zeekx/build_dl_framework/blob/dev/steps/step12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backward: vector
[x0, x1, ...] -> f -> [y0, y1, ...]

for y in f.outputs:

    y.grad -> f.backward -> [x0.grad, x1.grad, ...]


In [ ]:
import numpy as np

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
          if not isinstance(data, np.ndarray):
            raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None


    def set_creator(self, func):
        self.creator = func


# a -> f(a) -> b
# Variable: b.creator -> f
# b.creator -> f -> f.input -> a

    def backward(self):
        fs = [self.creator]
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        while fs:
            f = fs.pop()
            ygs = [o.grad for o in f.outputs]
            xgs = f.backward(*ygs)
            if not isinstance(xgs, tuple):
                xgs = (xgs,)
    
            for x, g in zip(f.inputs, xgs):
                x.grad = g
                if x.creator is not None:
                    fs.append(x.creator)


In [ ]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [ ]:
class Function:
    def __call__(self, *inputs): # input:[x0, x1, ...]@Variable -> [y0, y1, ...]@Variable
        self.inputs = inputs
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #unwrap
        if not isinstance(ys, tuple):
            ys = (ys,)

        self.outputs = [Variable(as_array(y)) for y in ys]
        for output in self.outputs:
            output.set_creator(self)

        return self.outputs if len(self.outputs) > 1 else self.outputs[0]

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


In [ ]:

class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.inputs[0].data
        gx = np.exp(x) * gy
        return gx


In [ ]:
class Add(Function):
    def forward(self, a, b):
        y = a + b
        return (y,)
    
    def backward(self, gy):
        return gy, gy

In [ ]:
class Identical(Function):
    def forward(self, x):
        return x
    
    def backward(self, gy):
        return gy # ??? 1 or gy

In [ ]:
def square(x):
  return Square()(x)

def exp(x):
  return Exp()(x)

def add(x0, x1):
  return Add()(x0, x1)

def identical(x):
  return Identical()(x)

Give a default grad value(array(1.0) for the last variable

In [ ]:
a = Variable(np.array(0.6))
b = Variable(np.array(0.4))
c = add(a, b)

c.backward()
a.grad, b.grad

In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))

a = square(x0)
b = identical(x1) # make sure a and b both have creator

y = add(a, b)

y.backward()

print(x0.grad, x1.grad)
print(x1.grad, x0.grad)
print(a.grad, b.grad)

In [ ]:
x3 = Variable(np.array(3.0))
x4 = Variable(np.array(4.0))

a, b = square(x3), square(x4) # make sure a and b both have creator
y = add(a, b)
y.backward()
print(y.data)
print(x3.grad, x4.grad)

In [ ]:
x =Variable(np.array(2))
y =Variable(np.array(3))
z = add(square(x) , square(y))
z. backward ()
print(z .data)
print(x .grad)
print(y .grad)
